In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
    # device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
set_session(session)

In [ ]:
from keras.models import load_model

model = load_model("gz_dev.model")
print(model.summary())

In [20]:
import random

from trainingHelper import TrainingHelper
from trainingGame import loadTrainingGame
import nnHelper
import numpy as np

from os import listdir
from os.path import isfile, join

filepath = "D:/GalconZero/Games/iteration_0"

BATCH_SIZE = 64

def getTrainingDataForNumGames(num):
    trainX = np.empty((0, 22, 14))
    trainY = [np.empty((0, 925)), np.empty((0,))]

    for _ in [1]:
        trainingGame = loadTrainingGame(filepath + "/game_1562210791.2397857_3.pickle")
        helper = TrainingHelper(trainingGame)    
        trainX = np.concatenate((trainX, helper.getTrainX()))
        [policy, value] = helper.getTrainY()
        trainY = [np.concatenate((trainY[0], policy)), np.concatenate((trainY[1], value))]

    print("Finished loading {} training files".format(num))
    return trainX, trainY

trainX, trainY = getTrainingDataForNumGames(1)

def getYAxisFlippedTrainingData(data):
    data = data.copy()
    for sample in data:
        for row in sample:
            row[7] *= -1
            row[10] *= -1
            row[13] *= -1
    return data

# this involves reversing ordering of each sample in trainX, then mapping indices somehow to fix trainY, both sends and redirects
def getXAxisFlippedTrainingData(data):
    pass


Finished loading 1 training files
[[ 0.49        0.16        0.47        0.          0.          1.
  -2.15       -0.          0.          0.         -0.          0.
   0.         -0.        ]
 [ 0.17        0.12        0.19        0.          0.          1.
  -1.65        0.57        0.          0.         -0.          0.
   0.         -0.        ]
 [ 0.79        0.21        0.06        0.          0.          1.
  -1.24        1.06        0.          0.         -0.          0.
   0.         -0.        ]
 [ 0.63        0.18        0.23        0.          0.          1.
  -1.18       -0.22        0.          0.         -0.          0.
   0.         -0.        ]
 [ 0.61        0.18        0.12        0.          0.          1.
  -1.06        0.54        0.2        -0.83        0.3515      0.
   0.         -0.        ]
 [ 0.35        0.14        0.3         0.          0.          1.
  -0.82        1.37        0.          0.         -0.          0.
   0.         -0.        ]
 [ 0.95     

In [12]:
historyPolicyValAcc = []
historyValueValAcc = []

historyPolicyValLoss = []
historyValueValLoss = []
historyValLoss = []

def doTrain(num_loops):
    for i in range(num_loops):
        history = trainingLoop()

        historyPolicyValAcc.extend(history.history['val_policy_acc'])
        historyValueValAcc.extend(history.history['val_value_acc'])
        
        historyPolicyValLoss.extend(history.history['val_policy_loss'])
        historyValueValLoss.extend(history.history['val_value_loss'])
        historyValLoss.extend(history.history['val_loss'])


In [ ]:
doTrain(20)

In [ ]:
import matplotlib.pyplot as plt 
    
plt.figure(1, figsize=(18, 18))  
   
# summarize history for accuracy  
   
plt.subplot(211)  
# plt.plot(history.history['policy_acc'])  
plt.plot(historyPolicyValAcc)  
# plt.plot(history.history['value_acc'])  
plt.plot(historyValueValAcc)  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
# plt.legend(['policy_categorical_accuracy', 'val_policy_categorical_accuracy', 'value_categorical_accuracy', 'val_value_categorical_accuracy'], loc='upper left')  
plt.legend(['Validation Policy Accuracy','Validation Value Accuracy'], loc='upper left')  
   
# summarize history for loss  
   
plt.subplot(212)  
# plt.plot(history.history['policy_loss'])  
plt.plot(historyPolicyValLoss)  
# plt.plot(history.history['value_loss'])  
plt.plot(historyValueValLoss)  
# plt.plot(history.history['loss'])  
plt.plot(historyValLoss)  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
# plt.legend(['policy_loss', 'val_policy_loss', 'value_loss','val_value_loss','loss','val_loss',], loc='upper left')  
plt.legend(['val_policy_loss','val_value_loss','val_loss'], loc='upper left')  

plt.show()  

In [ ]:
model.save("gz_dev.model")